In [48]:
# We start by importing necessary libraries.
import pathlib
import sys
import glob
import pandas as pd
import geopandas as gpd
# Here's the popexposure import 
import popexposure  

In [49]:
base_path = pathlib.Path.cwd().parent
data_dir = base_path / "demo_data"

In [50]:
# read in US wildfire dataset
fires = gpd.read_file(data_dir / "01_raw_data"/ "wfbz_disasters_conus.geojson")
# filter to only CA fires - wildfire_states has to contain CA
fires = fires[fires['wildfire_states'].str.contains('CA')]

# read in US wildfire dataset
fires = gpd.read_file(data_dir / "01_raw_data"/ "wfbz_disasters_conus.geojson")
# filter to only CA fires - wildfire_states has to contain CA
fires = fires[fires['wildfire_states'].str.contains('CA')]

In [ ]:
border_fires = fires[fires['wildfire_id'].str.contains('camp', case=False, na=False)].copy()

In [52]:
border_fires

,wildfire_id,wildfire_year,wildfire_states,wildfire_area,wildfire_complex,wildfire_complex_names,wildfire_total_fatalities,wildfire_max_civil_fatalities,wildfire_civil_fatalities,wildfire_struct_destroyed,...,wildfire_poo_lon,geometry_src,redbook_id,ics_id,mtbs_id,usgs_id,geomac_id,fema_id,fired_id,geometry
554,2004-09-13_BORDER_CA_SAN DIEGO_555,2004.0,CA,6.79688,False,None,0.0,0.0,NaN,1.0,...,-116.5509,mtbs,None,2004_CA-MVU-7603_BORDER,CA3258511651420040913,None,None,None,None,"MULTIPOLYGON (((-1899961.244 -343112.794, -189..."
711,2005-10-05_BORDER 50_CA_SAN DIEGO_712,2005.0,CA,14.37581,False,None,0.0,0.0,NaN,0.0,...,-116.6297,mtbs,None,2005_CA-MVU-008488_BORDER #50,CA3257711672220051005,None,None,FM-2585-CA,None,"MULTIPOLYGON (((-1914411.091 -342075.145, -191..."
4255,2016-06-19_BORDER 3_CA_SAN DIEGO_4256,2016.0,CA,32.20506,False,None,2.0,2.0,2.0,16.0,...,-116.64,mtbs,MVU-14498,2016_4249863_BORDER 3,CA3260811664120160619,None,None,FM-5128-CA,None,"MULTIPOLYGON (((-1898923.414 -337370.61, -1898..."


In [ ]:
border_fires["buffer_dist_10"] = 0  # buffer distance in meters 
border_fires.head()

,wildfire_id,wildfire_year,wildfire_states,wildfire_area,wildfire_complex,wildfire_complex_names,wildfire_total_fatalities,wildfire_max_civil_fatalities,wildfire_civil_fatalities,wildfire_struct_destroyed,...,geometry_src,redbook_id,ics_id,mtbs_id,usgs_id,geomac_id,fema_id,fired_id,geometry,buffer_dist_10
554,2004-09-13_BORDER_CA_SAN DIEGO_555,2004.0,CA,6.79688,False,None,0.0,0.0,NaN,1.0,...,mtbs,None,2004_CA-MVU-7603_BORDER,CA3258511651420040913,None,None,None,None,"MULTIPOLYGON (((-1899961.244 -343112.794, -189...",30000
711,2005-10-05_BORDER 50_CA_SAN DIEGO_712,2005.0,CA,14.37581,False,None,0.0,0.0,NaN,0.0,...,mtbs,None,2005_CA-MVU-008488_BORDER #50,CA3257711672220051005,None,None,FM-2585-CA,None,"MULTIPOLYGON (((-1914411.091 -342075.145, -191...",30000
4255,2016-06-19_BORDER 3_CA_SAN DIEGO_4256,2016.0,CA,32.20506,False,None,2.0,2.0,2.0,16.0,...,mtbs,MVU-14498,2016_4249863_BORDER 3,CA3260811664120160619,None,None,FM-5128-CA,None,"MULTIPOLYGON (((-1898923.414 -337370.61, -1898...",30000


In [54]:
# First, select cols
by_year = border_fires[["wildfire_id", "buffer_dist_10", "geometry"]]
# Then rename the wildfire ID col
by_year = by_year.rename(columns={"wildfire_id": "ID_hazard"})

In [55]:
by_year

,ID_hazard,buffer_dist_10,geometry
554,2004-09-13_BORDER_CA_SAN DIEGO_555,30000,"MULTIPOLYGON (((-1899961.244 -343112.794, -189..."
711,2005-10-05_BORDER 50_CA_SAN DIEGO_712,30000,"MULTIPOLYGON (((-1914411.091 -342075.145, -191..."
4255,2016-06-19_BORDER 3_CA_SAN DIEGO_4256,30000,"MULTIPOLYGON (((-1898923.414 -337370.61, -1898..."


In [56]:
# save by year in the data
border_fires = data_dir / "02_interim_data" / "border_fire.geojson"
by_year.to_file(border_fires, driver="GeoJSON")

In [57]:
# run popexp on it
ghsl_path = data_dir / "01_raw_data" / "GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0_R5_C8.tif"
# ZCTA path 
zcta_path = glob.glob(str(data_dir / "02_interim_data" / "*zcta*"))[0]

In [58]:
est = popexposure.PopEstimator()

year_dat = est.prep_data(path_to_data=border_fires, geo_type='hazard')
num_exposed = est.est_exposed_pop(pop_path=ghsl_path, 
                                  hazards = year_dat,
                                  hazard_specific=True)

In [59]:
num_exposed

,ID_hazard,exposed_10
0,2004-09-13_BORDER_CA_SAN DIEGO_555,0
1,2005-10-05_BORDER 50_CA_SAN DIEGO_712,0
2,2016-06-19_BORDER 3_CA_SAN DIEGO_4256,0
